In [50]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import re
import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', None)

In [51]:
SCORE_DIR = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores'

In [55]:
box_scores = os.listdir(SCORES_DIR)

In [56]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html") and (f.startswith("20240402"))] #or f.startswith("201710")or f.startswith("201711")or f.startswith("201712"))]

In [58]:
box_scores

['/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020SAC.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020PHI.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020UTA.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020MIA.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020GSW.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020MIN.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020TOR.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020DEN.html',
 '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores/202404020WAS.html']

In [59]:
def parse_html(box_scores):
    with open(box_scores) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    
    return soup

In [60]:
def read_line_score(soup):
    html = StringIO(str(soup))
    line_score = pd.read_html(html, attrs = {"id": "line_score"})[0]
    columns = list(line_score.columns)
    columns[0] = "Teams"
    columns[-1] = "Total"
    line_score.columns = columns 
    
    line_score = line_score[["Teams", "Total"]]
    return line_score

In [61]:
def four_factors(soup):
    html = StringIO(str(soup))
    factors = pd.read_html(html, attrs = {"id": "four_factors"})[0]
    columns = list(factors.columns)
    factors.columns = columns
    
    factors = factors[['Pace', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'ORtg']]
    return factors

In [62]:
def read_stats(soup, team, stat):
    html = StringIO(str(soup))
    stats = pd.read_html(html, attrs = {"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    stats = stats.apply(pd.to_numeric, errors = "coerce")
    return stats

In [63]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [64]:
def read_record(soup):
    he = []
    html = StringIO(str(soup))
    scoreboxes = soup.find_all(class_='scorebox')
    score_pattern = re.compile(r'\d+-\d+')
    
    for scorebox in scoreboxes:
        # Find all div elements within each scorebox that match the score pattern
        score_divs = scorebox.find_all('div', string=score_pattern)
        # If any matching divs are found
        if score_divs:
            # Loop through each matching div
            for score_div in score_divs:
                # Do something with the div
                record = score_div.text.split("-")
                df_record = pd.DataFrame([record])
                he.append(df_record)
    df = pd.concat(he)
    columns = list(df.columns)
    columns[0] = "Wins"
    columns[1] = "Losses"
    df.columns = columns
    df = df.reset_index()
    del df["index"]
    return df

In [65]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    factors = four_factors(soup)
    try:
        records = read_record(soup)
    except:
        continue
    teams = list(line_score['Teams'])
    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        advanced.drop("MP", axis=1, inplace=True)
        total = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        total.index = total.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "maxes"

        summary = pd.concat([total, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]
        summaries.append(summary)

    summary = pd.concat(summaries, axis =1).T

    game = pd.concat([summary, line_score,factors, records], axis = 1)
    game['home'] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis = 1)

    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")
    full_game["won"] = full_game["Total"] > full_game["Total_opp"]

    games.append(full_game)

    if (len(games) % 100 == 0):
        print(f"{len(games)} / {len(box_scores)}")

In [66]:
green_eggs = pd.concat(games, ignore_index = True)

In [67]:
def setup(df, whole):
    date = check['date'].iloc[-1][:10]
    before = pd.to_datetime(date)
    #after = pd.Timestamp(datetime.date(2024,12,12))

     # Subtract a year from 'date' column values exceeding the threshold
    for index,row in df.iterrows():
        if row['date'] <= before:
            df.drop(labels=index, axis=0, inplace=True)
    
#     for index,row in df.iterrows():
#         if row['date'] > after:
#             df.drop(labels=index, axis=0, inplace=True)
            
    df = df.sort_values("date")
    df = df.reset_index(drop=True)
    
    del df['index_opp']
    del df["mpmaxes"]
    del df["mpmaxes_opp"]
    del df['+/-']
    del df["+/-_opp"]
    
    return df

In [68]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/"
csv_path = "NBA_2018_2024.csv"
complete_path = folder_path + csv_path
whole = pd.read_csv(complete_path, index_col=0)

In [69]:
whole

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fgmaxes,fgamaxes,fg%maxes,3pmaxes,3pamaxes,3p%maxes,ftmaxes,ftamaxes,ft%maxes,orbmaxes,drbmaxes,trbmaxes,astmaxes,stlmaxes,blkmaxes,tovmaxes,pfmaxes,ptsmaxes,+/-maxes,ts%maxes,efg%maxes,3parmaxes,ftrmaxes,orb%maxes,drb%maxes,trb%maxes,ast%maxes,stl%maxes,blk%maxes,tov%maxes,usg%maxes,ortgmaxes,drtgmaxes,Teams,Total,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Wins,Losses,home,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fgmaxes_opp,fgamaxes_opp,fg%maxes_opp,3pmaxes_opp,3pamaxes_opp,3p%maxes_opp,ftmaxes_opp,ftamaxes_opp,ft%maxes_opp,orbmaxes_opp,drbmaxes_opp,trbmaxes_opp,astmaxes_opp,stlmaxes_opp,blkmaxes_opp,tovmaxes_opp,pfmaxes_opp,ptsmaxes_opp,+/-maxes_opp,ts%maxes_opp,efg%maxes_opp,3parmaxes_opp,ftrmaxes_opp,orb%maxes_opp,drb%maxes_opp,trb%maxes_opp,ast%maxes_opp,stl%maxes_opp,blk%maxes_opp,tov%maxes_opp,usg%maxes_opp,ortgmaxes_opp,drtgmaxes_opp,Teams_opp,Total_opp,Pace_opp,eFG%_opp,TOV%_opp,ORB%_opp,FT/FGA_opp,ORtg_opp,Wins_opp,Losses_opp,home_opp,season,date,won
0,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,9.0,37.0,46.0,24.0,11.0,4.0,10.0,24.0,99.0,0.500,0.455,0.364,0.284,18.0,80.4,47.9,66.7,11.1,6.6,9.2,100.0,99.7,102.7,11.0,23.0,1.000,4.0,9.0,0.500,5.0,7.0,1.00,4.0,9.0,10.0,10.0,4.0,2.0,3.0,5.0,25.0,8.0,0.798,1.000,1.000,2.000,10.5,26.8,13.7,46.5,9.9,4.5,34.7,29.9,129.0,112.0,BOS,99,99.3,0.455,9.2,18.0,0.216,99.7,0,1,0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,100.0,102.7,99.7,12.0,19.0,0.667,1.0,5.0,0.333,6.0,7.0,1.00,3.0,15.0,16.0,9.0,2.0,2.0,4.0,4.0,29.0,9.0,0.698,0.667,0.500,1.000,11.0,35.0,19.4,43.7,3.8,6.0,31.6,27.3,138.0,107.0,CLE,102,99.3,0.488,15.3,19.6,0.253,102.7,1,0,1,2018,2017-10-17,False
1,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,9.0,41.0,50.0,19.0,3.0,4.0,17.0,25.0,102.0,0.543,0.488,0.265,0.301,19.6,82.0,52.1,50.0,3.0,7.1,15.3,100.0,102.7,99.7,12.0,19.0,0.667,1.0,5.0,0.333,6.0,7.0,1.00,3.0,15.0,16.0,9.0,2.0,2.0,4.0,4.0,29.0,9.0,0.698,0.667,0.500,1.000,11.0,35.0,19.4,43.7,3.8,6.0,31.6,27.3,138.0,107.0,CLE,102,99.3,0.488,15.3,19.6,0.253,102.7,1,0,1,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,9.0,37.0,46.0,24.0,11.0,4.0,10.0,24.0,99.0,0.500,0.455,0.364,0.284,18.0,80.4,47.9,66.7,11.1,6.6,9.2,100.0,99.7,102.7,11.0,23.0,1.000,4.0,9.0,0.500,5.0,7.0,1.00,4.0,9.0,10.0,10.0,4.0,2.0,3.0,5.0,25.0,8.0,0.798,1.000,1.000,2.000,10.5,26.8,13.7,46.5,9.9,4.5,34.7,29.9,129.0,112.0,BOS,99,99.3,0.455,9.2,18.0,0.216,99.7,0,1,0,2018,2017-10-17,True
2,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,0.684,10.0,33.0,43.0,28.0,9.0,5.0,12.0,16.0,122.0,0.579,0.562,0.423,0.196,22.2,84.6,51.2,59.6,8.8,10.0,10.2,100.0,119.6,118.6,10.0,23.0,0.667,4.0,9.0,0.667,6.0,8.0,0.75,3.0,7.0,8.0,11.0,2.0,2.0,3.0,4.0,27.0,20.0,0.859,0.889,0.667,0.667,9.6,26.1,13.8,42.9,2.8,6.5,18.2,31.2,152.0,126.0,HOU,122,102.0,0.562,10.2,22.2,0.134,119.6,1,0,0,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,100.0,118.6,119.6,8.0,18.0,1.000,6.0,9.0,0.857,4.0,5.0,1.00,1.0,10.0,11.0,13.0,2.0,4.0,8.0,4.0,23.0,11.0,1.218,1.222,0.778,1.000,13.1,38.6,22.7,57.1,6.1,9.1,31.7,32.6,250.0,127.0,GSW,121,102.0,0.638,16.0,15.4,0.238,118.6,0,1,1,2018,2017-10-17,True
3,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,6.0,35.0,41.0,34.0,5.0,9.0,17.0,25.0,121.0,0.678,0.638,0.375,0.263,15.4,77.8,48.8,79.1,4.9,16.1,16.0,100.0,118.6,119.6,8.0,18.0,1.000,6.0,9.0,0.857,4.0,5.0,1.00,1.0,10.0,11.0,13.0,2.0,4.0,8.0,4.0,23.0,11.0,1.218,1.222,0.778,1.000,13.1,38.6,22.7,57.1,6.1,9.1,31.7,32.6,250.0,

In [70]:
df = setup(green_eggs, whole)
complete = pd.concat([whole,df], axis=0)

In [71]:
complete['date']

0               2017-10-17
1               2017-10-17
2               2017-10-17
3               2017-10-17
4               2017-10-18
5               2017-10-18
6               2017-10-18
7               2017-10-18
8               2017-10-18
9               2017-10-18
10              2017-10-18
11              2017-10-18
12              2017-10-18
13              2017-10-18
14              2017-10-18
15              2017-10-18
16              2017-10-18
17              2017-10-18
18              2017-10-18
19              2017-10-18
20              2017-10-18
21              2017-10-18
22              2017-10-18
23              2017-10-18
24              2017-10-18
25              2017-10-18
26              2017-10-19
27              2017-10-19
28              2017-10-19
29              2017-10-19
              ...         
166    2024-04-01 00:00:00
167    2024-04-01 00:00:00
168    2024-04-01 00:00:00
169    2024-04-01 00:00:00
170    2024-04-01 00:00:00
171    2024-04-01 00:00:00
1

In [72]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/"
csv_path = "NBA_2018_2024.csv"
complete_path = folder_path + csv_path
complete.to_csv(complete_path)